In [22]:
import numpy as np
import pandas as pd
import mne
from mne_bids import (BIDSPath, read_raw_bids, find_matching_paths)
import xarray as xr
from ptsa.data.timeseries import TimeSeries
from ptsa.data.concat import concat
import os
# import cmldask.CMLDask as da
# import cmlreaders as cml
import pickle

def mne_to_ptsa(ep):
    '''Create an PTSA TimeSeries (essentially xarray) version of MNE epoch data'''
    assert ep.metadata is not None, "Please define mne.Epoch.metadata"
    x = TimeSeries(ep.get_data(copy=True), dims=('event','channel','time'),
                   coords={'event':pd.MultiIndex.from_frame(ep.metadata),
                           'channel':ep.info['ch_names'],
                           'time':ep.times,
                           'samplerate':ep.info['sfreq']})
    return x

In [2]:
subject = 'LTP453'
settings_path = "settings/phase1_settings.pkl"
settings = pickle.load(open(settings_path, 'rb'))
# data = cml.get_data_index(kind = 'ltp')
# data = data[(data['experiment']==settings['experiment'])&(data['subject']==subject)].sort_values('session').reset_index()

In [3]:
settings

{'rel_start': 300,
 'rel_stop': 1300,
 'buffer_time': 500,
 'freqs': array([  6.        ,   9.75368156,  15.85571732,  25.77526961,
         41.90062864,  68.11423148, 110.72742057, 180.        ]),
 'width': 5,
 'experiment': 'NiclsCourierReadOnly',
 'clean': 0,
 'save': 1,
 'reference': 'average'}

In [15]:
with open('settings/phase2_settings_early_prestimulus.pkl', 'wb') as f:
    pickle.dump(settings.__dict__, f)

In [31]:
bids_root = "~/data/nicls_bids"
bids_root = os.path.expanduser(bids_root)
bids_paths = find_matching_paths(
    bids_root, 
    subjects=subject, 
    tasks="NiclsCourierReadOnly",
    datatypes="eeg",
    extensions=".bdf"
    )
all_sessions = []
for bids_path in bids_paths:
    events_path = bids_path.copy().update(suffix="events", extension=".tsv")
    event_df = pd.read_csv(events_path, sep="\t")
    raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
    mne_events, mne_event_id = mne.events_from_annotations(raw, verbose=False)
    epochs = mne.Epochs(
        raw,
        mne_events, 
        event_id={'WORD':mne_event_id['WORD']}, 
        event_repeated='drop', 
        tmin=(settings['rel_start']-settings['buffer_time'])/1000.,
        tmax=(settings['rel_stop']+settings['buffer_time'])/1000., 
        baseline=None, 
        preload=True
    )
    epochs.metadata = event_df.query("trial_type=='WORD'").reset_index(drop=True)
    # do more stuff here
    all_sessions.append(mne_to_ptsa(epochs))
all_sessions = concat(all_sessions, 'event')
    

Not setting metadata
120 matching events found
No baseline correction applied
0 projection items activated
Loading data for 120 events and 4097 original time points ...


/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_27445/1687044452.py:14: RuntimeWarning: The unit for channel(s) EXG5, EXG6, EXG7, EXG8 has changed from V to NA.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_27445/1687044452.py:14: RuntimeWarning: The unit for channel(s) Status has changed from NA to V.
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_27445/1687044452.py:14: RuntimeWarning: There are channels without locations (n/a) that are not marked as bad: ['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 'Status']
  raw = read_raw_bids(bids_path, extra_params={"infer_types":True}, verbose=False)
/var/folders/7h/662tdm8d6sn0krrht717wzmm0000gq/T/ipykernel_27445/1687044452.py:14: RuntimeWarning: Not setting positions of 9 eog/misc/stim channels found in montage:
['EXG1', 'E

0 bad epochs dropped
Adding metadata with 15 columns


In [27]:
settings['freqs']

array([  6.        ,   9.75368156,  15.85571732,  25.77526961,
        41.90062864,  68.11423148, 110.72742057, 180.        ])

In [9]:
import cmlreaders as cml
data = cml.get_data_index(kind = 'ltp')
data = data[(data['experiment']==settings['experiment'])&(data['subject']==subject)].sort_values('session').reset_index()
# cml.CMLReader(subject=subject, experiment=row['experiment'], session=row['session'])
r = cml.CMLReader(subject, experiment=settings['experiment'], session=0)
evs = r.load('task_events')
word_evs = evs[(evs.type=='WORD')&(evs.eegoffset!=-1)]
if len(word_evs)==0:
    # continue # sync pulses not recorded
    print("bad!")
eeg = r.load_eeg(word_evs,
                    rel_start=settings['rel_start'] - settings['buffer_time'],
                    rel_stop=settings['rel_stop'] + settings['buffer_time'],
                    clean=settings['clean']
                ).to_ptsa()

NameError: name 'cml' is not defined

In [14]:
assert (eeg.item == all_sessions[0].item).all()
assert (all_sessions[0]['item'].values == eeg['item'].values).all()

AttributeError: 'bool' object has no attribute 'all'